# **Building ReAct Agents**

## **What's Covered?**
1. Introduction to Agent
    - What is an Agent?
    - What is a ReAct Agent?
    - Understanding the ReAct Loop
    - What is Langchain's create_agent()?
    - Implementing an Agent with Core Component - Model, Tools and Prompt
    - A simple implementation - Input Output
2. Implementing an Agent
    - Agent's Core Component - Model, Tools and Prompt
    - Step 1: Init Chat Models
    - Step 2: Init an Agent
    - Step 3: Invoking the Agent
    - Step 4: Understanding Models Response
    - Step 5: Pretty Printing the Model's Response
3. Enforcing Structured Output
    - Introduction to Structured Output
    - Implementing Structured Responses
    - Step 1: Defining the Schema for response_format
    - Step 2: Agent Init with ProviderStrategy for Structured Output
    - Step 3: Agent Init with Invoking the Agent and Printing structured_response
    - Step 4: ToolStrategy for Structured Output
    - Passing Multiple Schemas
4. Error Handling with ToolStrategy
    - Multiple Structured Outputs Error
    - Schema Validation Error
    - Custom Errors (Coming Soon)
5. Streaming Agent's Response
    - Streaming Agent Progress
    - Streaming LLM Response
    - Streaming Multiple Modes

## **Introduction to Agent**

### **What is an Agent?**
Agents combine language models with tools to create systems that can reason about tasks, decide which tools to use, and iteratively work towards solutions.

An LLM Agent runs tools in a loop to achieve a goal. An agent runs until a stop condition is met - i.e., when the model emits a final output or an iteration limit is reached.

### **What is a ReAct Agent**?
ReAct is an agent pattern where the LLM alternates between Reason (think) and Act (use tools) in a loop until it solves the task.
In short, 
> **Agent = Reasoning and Decision Making (using LLMs) + Action (using Tools)**

### **Understanding the ReAct Loop**
Imagine asking an agent: "What's the weather in SF and book a flight if sunny?"
```
1. Reason: "First check weather. Use get_weather tool."
   ↓ (calls tool)
2. Act: Tool runs → "SF: Sunny 72°F"
   ↓ (feeds back)
3. Reason: "Sunny! Now book flight. Use book_flight tool."
   ↓ (calls tool)
4. Act: Tool runs → "Flight booked: UA123"
   ↓
5. Finish: "Weather sunny. Flight booked."
```

### **What is Langchain's create_agent()?**
In Langchain **create_agent()** provides a production-ready agent implementation.

In create_agent(): Built-in ReAct—model reasons, picks tools, loops automatically.

Introduced in LangChain 1.x, **create_agent()** is a graph-based ReAct agent (built on LangGraph) that follows **model → tools → model** loops until completion.


**create_agent** builds a graph-based agent runtime using LangGraph. A graph consists of nodes (steps) and edges (connections) that define how your agent processes information. The agent moves through this graph, executing nodes like the model node (which calls the model), the tools node (which executes tools), or middleware.

## **Implementing an Agent**

### **Agent's Core Component - Model, Tools and Prompt**

Every agent needs a model, tools (optional), and system_prompt (optional).

**Syntax**
```python
from langchain.agents import create_agent
from langchain.tools import tool

@tool
def check_weather():
    "This check the weather"
    return "Hot"

agent = create_agent(
    model=llm,
    tools=[check_weather],
    system_prompt="You are a helpful assistant with weather tools."  # Shapes behavior
)
```

### **Step 1: Init Chat Models**

In [1]:
# Setup API Key

f = open('keys/.gemini.txt')

GOOGLE_API_KEY = f.read()

In [2]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [3]:
# Setup API Key

f = open('keys/.groq_api_key.txt')

GROQ_API_KEY = f.read()

In [5]:
# Import ChatModel
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# Pass the standard parameters during initialization
google_chat_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, 
                                           model="gemini-2.5-flash", 
                                           temperature=1)

openai_chat_model = ChatOpenAI(api_key=OPENAI_API_KEY, 
                               model="gpt-4o-mini", 
                               temperature=1)

groq_chat_model = ChatGroq(api_key=GROQ_API_KEY, 
                           model="openai/gpt-oss-20b", 
                           temperature=1, 
                           )

### **Step 2: Init an Agent**

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=openai_chat_model,
    system_prompt="You are a helpful AI assistant"
)

### **Step 3: Invoking the Agent**

In [8]:
result = agent.invoke({ 
    "messages": [{"role": "human", "content": "Hi, My name is ThatAIGuy."}] 
})

print(result)

{'messages': [HumanMessage(content='Hi, My name is ThatAIGuy.', additional_kwargs={}, response_metadata={}, id='b1e16438-52d3-48c7-8f94-8a86a2c1b7bc'), AIMessage(content='Hi ThatAIGuy! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 27, 'total_tokens': 40, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CvcJwyz4w0Oo3vHY6B7giUthaRPP4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--c1dd92d1-a883-410b-9cb2-d80dde52dfaa-0', usage_metadata={'input_tokens': 27, 'output_tokens': 13, 'total_tokens': 40, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio

### **Step 4: Understanding Models Response**

The Agents response will be a key value pair with following keys:
1. **messages:** Always present in every agent response. **messages** contains the complete conversation history including user inputs, model reasoning, tool calls, and tool results. Each message can be displayed using .pretty_print() for readable formatting.
2. **structured_response:** Parsed and validated output that matches your defined schema. Available only when you specify a **response_format** parameter inside **create_agent()**. Automatically handles validation errors and retries.

In [9]:
print(result.keys())

dict_keys(['messages'])


### **Step 5: Pretty Printing the Model's Response**

In [10]:
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Hi, My name is ThatAIGuy.
================================== Ai Message ==================================

Hi ThatAIGuy! How can I assist you today?


## **Enforcing Structured Output**

### **Introduction to Structured Output**
Structured output allows agents to return data in a specific, predictable format. Instead of parsing natural language responses, you get structured data in the form of JSON objects, Pydantic models, or dataclasses that your application can directly use. 


### **Implementing Structured Responses** 
We can enforce structured output within **create_agent()** using **response_format** to return validated data in a specific schema (Pydantic/JSON) instead of free-form text.

**response_format** parameter inside **create_agent()** controls how the agent returns structured data. To apply structured output, you should either use ProviderStrategy (default) or ToolStrategy.

- **ProviderStrategy**: Some model providers support structured output natively through their APIs (e.g. OpenAI, Grok, Gemini). This is the most reliable method when available.
- **ToolStrategy**: For models that don’t support native structured output, LangChain uses tool calling to achieve the same result. This works with all models that support tool calling, which is most modern models.


**Syntax**
```python
from langchain.agents.structured_output import ProviderStrategy, ToolStrategy
```

### **Step 1: Defining the Schema for response_format**

In [11]:
from pydantic import BaseModel, Field

class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

### **Step 2: Agent Init with ProviderStrategy for Structured Output**

Some model providers support structured output natively through their APIs (e.g. OpenAI, Grok, Gemini). This is the most reliable method when available.

```python
class ProviderStrategy(Generic[SchemaT]):
    schema: type[SchemaT]
    strict: bool | None = None
```

- **schema:** The schema defining the structured output format.
- **strict:** Optional boolean parameter to enable strict schema adherence. Supported by some providers (e.g., OpenAI and xAI). Defaults to None (disabled).

LangChain automatically uses ProviderStrategy when you pass a schema type directly to create_agent.response_format and the model supports native structured output:

```python
agent = create_agent(
    model=openai_chat_model,
    response_format=ContactInfo,  # Auto-selects ProviderStrategy
    system_prompt="""You are a helpful assistant. 
                     Be concise and accurate. 
                     You are suppose to extract contact info from the user inputs.
                     """
)
```

In [12]:
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model=openai_chat_model,
    response_format=ProviderStrategy(schema=ContactInfo),
    system_prompt="""You are a helpful assistant. 
                     Be concise and accurate. 
                     You are suppose to extract contact info from the user inputs.
                     """
)

### **Step 3: Invoking the Agent and Printing structured_response**

LangChain’s **create_agent** handles structured output automatically. The user sets their desired structured output schema, and when the model generates the structured data, it’s captured, validated, and **returned in the `'structured_response'` key** of the agent’s state.

In [13]:
result = agent.invoke({
    "messages": [{"role": "human", 
                  "content": "My name is John Doe, email john@example.com, and number (555) 123-4567"}]
})

print(result["structured_response"])

name='John Doe' email='john@example.com' phone='(555) 123-4567'


### **Step 4: Agent Init with ToolStrategy for Structured Output**

For models that don’t support native structured output, LangChain uses tool calling to achieve the same result. This works with all models that support tool calling, which is most modern models.

```python
class ToolStrategy(schema):
    schema: schema,
    tool_message_content: str | None,
    handle_errors: ERROR_HANDLING_LOGIC
```

- **schema:** The schema defining the structured output format.
- **tool_message_content:** The tool_message_content parameter allows you to customize the message that appears in the conversation history when structured output is generated as per the given schema.
- **handle_errors:** Error handling strategy for structured output validation failures. By default ERROR_HANDLING_LOGIC is set to boolean value True.

In [14]:
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=groq_chat_model,
    response_format=ToolStrategy(schema=Union[ContactInfo,]),  # Default: handle_errors=True
    system_prompt="""You are a helpful assistant. 
                     Be concise and accurate. 
                     You are suppose to extract contact info from the user inputs.
                     """
)

In [15]:
result = agent.invoke({
    "messages": [{"role": "human", "content": "My name is John Doe, email john@example.com, and number (555) 123-4567"}]
})

print(result["structured_response"])

name='John Doe' email='john@example.com' phone='(555) 123-4567'


### **Important: Passing Multiple Schemas** 
1. The `Union[ContactInfo,]` enables multiple possible schemas—the model picks the best match (or you get type errors if invalid).
2. Without Union, only one exact schema allowed. `Union[ContactInfo,]` (note trailing comma) is a single-item Union—technically valid Python but redundant here.
3. Agent auto retries if:
    - Wrong schema picked
    - Validation fails (eg: missing email id)

## **Error Handling**

### **Multiple structured outputs error**

Models can make mistakes when generating structured output via tool calling. 

Sometimes cheap llms (like gpt-4o-mini) can greedily calls 2+ schemas. This is rare with good models/prompts.

LangChain provides intelligent retry mechanisms to handle these errors automatically.

In [16]:
from pydantic import BaseModel, Field

class ContactInfo(BaseModel):
    name: str = Field(description="Person's name")
    email: str = Field(description="Email address")

class EventDetails(BaseModel):
    event_name: str = Field(description="Name of the event")
    date: str = Field(description="Event date")

In [17]:
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=openai_chat_model,
    tools=[],
    response_format=ToolStrategy(Union[ContactInfo, EventDetails], handle_errors=True),
)

In [19]:
response = agent.invoke({
    "messages": [{"role": "user", 
                  "content": "Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th"}]
})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th
================================== Ai Message ==================================
Tool Calls:
  ContactInfo (call_rO6jYtxfL8EbI9R9RnpggZcw)
 Call ID: call_rO6jYtxfL8EbI9R9RnpggZcw
  Args:
    name: John Doe
    email: john@email.com
  EventDetails (call_coxQK7I126SfQTxoNSn8sdBV)
 Call ID: call_coxQK7I126SfQTxoNSn8sdBV
  Args:
    event_name: Tech Conference
    date: March 15th
================================= Tool Message =================================
Name: ContactInfo

Error: Model incorrectly returned multiple structured responses (ContactInfo, EventDetails) when only one is expected.
 Please fix your mistakes.
================================= Tool Message =================================
Name: EventDetails

Error: Model incorrectly returned multiple structured responses (ContactInfo, EventDetails) when only one i

### **Important Consideration**
**Note:** This can increase the token consumption.

**Idea:** Refactor Schema Handling instead of juggling multiple schemas, split the process into a parallel workflow. Create two distinct nodes, where each node is responsible for a single response_format.

### **Schema validation error**

When structured output doesn’t match the expected schema, the agent provides specific error feedback.

In [20]:
from pydantic import BaseModel, Field

class ProductRating(BaseModel):
    rating: int | None = Field(description="Rating from 1-5", ge=1, le=5)
    comment: str = Field(description="Review comment")

In [21]:
agent = create_agent(
    model=openai_chat_model,
    tools=[],
    response_format=ToolStrategy(ProductRating),  # Default: handle_errors=True
    system_prompt="""You are a helpful assistant that parses product reviews and ratings. 
                     Parse EXACTLY as user says. Do NOT fix values. Even if user gives rating >5.
                     Do not make any false field or false value."""
)

In [22]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "User mentioned Amazing product, 10/10!"}]
})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

User mentioned Amazing product, 10/10!
================================== Ai Message ==================================
Tool Calls:
  ProductRating (call_pvMHTN5VZzF2TneGaFM2gEp4)
 Call ID: call_pvMHTN5VZzF2TneGaFM2gEp4
  Args:
    rating: 10
    comment: Amazing product, 10/10!
================================= Tool Message =================================
Name: ProductRating

Error: Failed to parse structured output for tool 'ProductRating': Failed to parse data to ProductRating: 1 validation error for ProductRating
rating
  Input should be less than or equal to 5 [type=less_than_equal, input_value=10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than_equal.
 Please fix your mistakes.
================================== Ai Message ==================================
Tool Calls:
  ProductRating (call_YA2ehDwXQhBf4LlbZ1TpO3eb)
 Call ID: call_YA2ehDwXQhBf4LlbZ1Tp

### **Important Consideration**

**ISSUE 1:** Some models will intentionally fix 10 to 5 at server-side because they knows your schema constraints (ge=1, le=5) from the description.
> **SOLUTION 1:** Use a strict prompt to enforce model to parse the EXACT values.

**ISSUE 2:** Groq does strict validation server-side. Due to this it will throw 400 error even before LangChain's `handle_error=True` can catch/convert to `ToolMessage` retry. 
> **SOLUTION 2:** We can disable this behaviour by passing `kwargs={"strict": "false"}`.

### **Custom Errors**

**Coming Soon**

## **Streaming Agent's Response**

LangChain’s streaming system lets you surface live feedback from agent runs to your application.

Following is possible with LangChain Streaming:
1. Stream agent progress
2. Stream LLM tokens
3. Stream multiple modes
4. Stream custom updates (Check LangChain docs)

**Syntax**
```python
for chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="updates",
):
    # do something
    pass
```

### **Streaming Agent Progress**

To stream agent progress, use the stream or astream methods with **stream_mode="updates"**. This emits an event after every agent step.

**Note:** Emit only the node or task names and updates returned by the nodes or tasks after each step. If multiple updates are made in the same step (e.g. multiple nodes are run) then those updates are emitted separately.

In [23]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""

    return f"It's always sunny in {city}!"

agent = create_agent(
    model=openai_chat_model,
    tools=[get_weather],
)

In [24]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "What is the weather in SF?"}]
})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What is the weather in SF?
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_NLss1oaAqhk5XFRNuDJwzPjB)
 Call ID: call_NLss1oaAqhk5XFRNuDJwzPjB
  Args:
    city: San Francisco
================================= Tool Message =================================
Name: get_weather

It's always sunny in San Francisco!
================================== Ai Message ==================================

The weather in San Francisco is sunny!


In [25]:
for chunk in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="updates",
):
    for step, data in chunk.items():
        print(f"step: {step}")
        print(f"content: {data['messages'][-1].content_blocks}")

step: model
content: [{'type': 'tool_call', 'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_YqNOFqTXjIv3GHaoOc8xJYns'}]
step: tools
content: [{'type': 'text', 'text': "It's always sunny in San Francisco!"}]
step: model
content: [{'type': 'text', 'text': 'The weather in San Francisco is sunny!'}]


### **Streaming LLM Tokens**

To stream tokens as they are produced by the LLM, use **stream_mode="messages"**. Below you can see the output of the agent streaming tool calls and the final response.

**Note:** Emit LLM messages token-by-token together with metadata for any LLM invocations inside nodes or tasks. Will be emitted as 2-tuples (LLM token, metadata).

In [28]:
for token, metadata in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")

node: model
content: [{'type': 'tool_call_chunk', 'id': 'call_GQR2M6tcmnqBjtXLSFg2a1GM', 'name': 'get_weather', 'args': '', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '{"', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': 'city', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '":"', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': 'San', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': ' Francisco', 'index': 0}]


node: model
content: [{'type': 'tool_call_chunk', 'id': None, 'name': None, 'args': '"}', 'index': 0}]


node: model
content: []


node: model
content: []


node: model
content: []


node: tools
content: [{'type': 'text', 'text': "It's always sunny in San Francisco!"}]


node: model
content: []


node: model
content: [{'ty

In [32]:
for token, metadata in agent.stream(  
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages",
):
    if token.content:
        print(token.content, end="", flush=True)

It's always sunny in San Francisco!The weather in San Francisco is sunny!

### **Stream Multiple Modes**

You can specify multiple streaming modes by passing stream mode as a list: `stream_mode=["updates", "custom"]`. The streamed outputs will be tuples of `(mode, chunk)` where mode is the name of the stream mode and chunk is the data streamed by that mode.

In [40]:
from langchain.messages import AIMessageChunk, AnyMessage, AIMessage, ToolMessage

def _render_message_chunk(token: AIMessageChunk) -> None:
    if token.text:
        print(token.text, end="|")
    if token.tool_call_chunks:
        print(token.tool_call_chunks)

def _render_completed_message(message: AnyMessage) -> None:
    if isinstance(message, AIMessage) and message.tool_calls:
        print(f"Tool calls: {message.tool_calls}")
    if isinstance(message, ToolMessage):
        print(f"Tool response: {message.content_blocks}")        

for stream_mode, data in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in Boston?"}]},
    stream_mode=["messages", "updates"],  
):
    if stream_mode == "messages":
        token, metadata = data
        if isinstance(token, AIMessageChunk):
            _render_message_chunk(token)
    if stream_mode == "updates":
        for source, update in data.items():
            if source in ("model", "tools"):
                _render_completed_message(update["messages"][-1])

[{'name': 'get_weather', 'args': '', 'id': 'call_uepRv0k5u8sNGQPUIueBCKnk', 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '{"', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': 'city', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '":"', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': 'Boston', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
[{'name': None, 'args': '"}', 'id': None, 'index': 0, 'type': 'tool_call_chunk'}]
Tool calls: [{'name': 'get_weather', 'args': {'city': 'Boston'}, 'id': 'call_uepRv0k5u8sNGQPUIueBCKnk', 'type': 'tool_call'}]
Tool response: [{'type': 'text', 'text': "It's always sunny in Boston!"}]
The| weather| in| Boston| is| always| sunny|!|